<a href="https://colab.research.google.com/github/HwangHanJae/BigDataPratice/blob/main/1_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형1

데이터 출처 : https://data.kma.go.kr/cmmn/static/staticPage.do?page=intro

데이터 설명 : 2020년도 이화동(서울) , 수영동(부산)의 시간단위의 기온과 강수량

데이터 URL : https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv

## Q1. 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개인가?


In [30]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")

#첫번째 여름철 분류하기
df['time'] = pd.to_datetime(df['time'])
#df['time'].dt.month
df_summer = df.loc[(df['time'].dt.month >= 6) & (df['time'].dt.month <= 8)]

#이화동 > 수영동 기온
time = df_summer['이화동기온'] > df_summer['수영동기온']
print("답 : ")
print(time.sum())

답 : 
1415


## Q2. 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라

In [47]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")
#최대 강수량
max1 = df["이화동강수"].max()
max2 = df["수영동강수"].max()

#시간대를 구하기
time1 = df.loc[df['이화동강수'] == df['이화동강수'].max(), 'time'].values
time2 = df.loc[df['수영동강수'] == df['수영동강수'].max(), 'time'].values
print(time1)
print(time2)

['2020-09-30 09:00:00']
['2020-07-23 12:00:00']


#작업형2

데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형

데이터 설명 :  고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)

문제타입 : 분류유형

평가지표 : f1-score

trainData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv

testData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/test.csv

subData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/submission.csv

##Baseline

In [96]:
trainData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv"
testData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/test.csv"
subData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/submission.csv"

#라이브러리로드
import pandas as pd

#데이터로드
train = pd.read_csv(trainData)
test = pd.read_csv(testData)
submission = pd.read_csv(subData)

#라벨인코더와 랜덤포레스트분류기
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier



#전처리함수
def preprocessing(df, target='train', encoderLst = None):

  #모델성능에 영향을 주지 않는 컬럼을 제거
  drop_col = ['RowNumber', "CustomerId", "Surname"]
  df = df.drop(drop_col, axis=1)

  #범주형 컬럼인 Geograpy와 Gender를 labelencoding을 해주고 인코더 testset에서 사용하기 위해 인코더리스트를 반환

  #훈련과 테스트에 대해서 다르게 적용
  if target == "train":
    labelEncoder1 = LabelEncoder()
    labelEncoder2 = LabelEncoder()

    df['Geography'] = labelEncoder1.fit_transform(df['Geography'])
    df['Gender'] = labelEncoder2.fit_transform(df['Gender'])
    encoderLst = [labelEncoder1, labelEncoder2]
    return df, encoderLst

  elif target == "test" and encoderLst != None:
    labelEncoder1 = encoderLst[0]
    labelEncoder2 = encoderLst[1]

    df['Geography'] = labelEncoder1.fit_transform(df['Geography'])
    df['Gender'] = labelEncoder1.fit_transform(df['Gender'])
    return df

  return None;

#전처리함수 실행
preprocessing_df, encoder_list = preprocessing(train)

#훈련데이터 분리
from sklearn.model_selection import train_test_split

X = preprocessing_df.drop('Exited',axis=1)
y= preprocessing_df['Exited']

#훈련진행 분류 모델이기 때문에 층화추출을 위해서 stratify 파라미터 설정
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 42, stratify = y)



#모델학습 및 평가
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

#f1score평가
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred))
#비교적 낮은 모델 성능 추가적인 전처리가 필요할듯

#제출 파일생성을 위해 test데이터에 대해서 전처리 진행
preprocessing_test_df = preprocessing(test, 'test', encoderLst=encoder_list)

test_pred = rf.predict(preprocessing_test_df)
submission = test_pred
print(submission)
#submission.to_csv('submission_model.csv', index=False)

0.5504151838671412
[0 0 0 ... 0 0 1]


baseline을 정리하자면

훈련에 필요없는 컬럼을 제거하여 정리하고

범주형 컬럼을 단순하게 정수값으로 바꾸어서 전치리를 진행하였음

랜덤포레스트분류기(RandomForestClassifier)을 사용하였고

평가지표는 f1-score를 사용했음.

## Pratice

In [168]:
trainData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv"
testData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/test.csv"
subData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/submission.csv"

#라이브러리로드
import pandas as pd

#데이터로드
train = pd.read_csv(trainData)
test = pd.read_csv(testData)
submission = pd.read_csv(subData)

#훈련데이터 정보

#print(train.info())

#필요없는 데이터 제거
drop_col = ["RowNumber","CustomerId","Surname"]
train = train.drop(drop_col, axis=1)
test = test.drop(drop_col, axis=1)

#범주형 데이터 바꾸기
#import sklearn.preprocessing
#help(sklearn.preprocessing)
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
columns = ["Geography", "Gender"]

for column in columns:
  for df in [train, test]:
    df[column] = label_encoder.fit_transform(df[column])

#데이터 나누기
from sklearn.model_selection import train_test_split

X = train.drop("Exited", axis=1)
y = train["Exited"]


X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42,stratify=y)
#모델링과 평가

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42,)
knn = KNeighborsClassifier()
lr = LogisticRegression()
tree = DecisionTreeClassifier()

models = [rf, knn, lr, gb,tree]
for model in models:
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  f1=f1_score(y_test, y_pred)
  print()
  print(model.__class__.__name__)
  print(f1)
  if model.__class__.__name__ == "GradientBoostingClassifier":
    y_pred = model.predict(test)
    print("완료")
    print(submission)
    submission["0"] = y_pred
    print()
    submission.to_csv("수험번호.csv",index=False)
    break;



RandomForestClassifier
0.5635528330781011

KNeighborsClassifier
0.09259259259259259

LogisticRegression
0.08315098468271334

GradientBoostingClassifier
0.5807453416149069
완료
        0
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
...   ...
1996  0.0
1997  0.0
1998  0.0
1999  0.0
2000  0.0

[2001 rows x 1 columns]

